# Caching Embeddings

Embeddings can be stored or temporarily cached to avoid needing to recompute them.

Caching embeddings can be done using a `CacheBackedEmbedder`.

The cache backed embedder is a wrapper around an embedder that caches
embeddings in a key-value store. 

The text is hashed and the hash is used as the key in the cache.


The main supported way to initialized a `CacheBackedEmbedder` is `from_bytes_store`. This takes in the following parameters:

- underlying_embedder: The embedder to use for embedding.
- document_embedding_cache: The cache to use for storing document embeddings.
- namespace: (optional, defaults to `""`) The namespace to use for document cache. This namespace is used to avoid collisions with other caches. For example, set it to the name of the embedding model used.

**Attention**: Be sure to set the `namespace` parameter to avoid collisions of the same text embedded using different embeddings models.

In [1]:
from langchain.embeddings import CacheBackedEmbedder
from langchain.storage import InMemoryStore
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings

## In Memory

This section shows how to set up an in memory cache for embeddings. This type of cache is primarily 
useful for unit tests or prototyping. Do **not** use this cache if you need to actually store the embeddings.

In [2]:
store = InMemoryStore()

In [3]:
underlying_embedder = OpenAIEmbeddings()
embedder = CacheBackedEmbedder.from_bytes_store(
    underlying_embedder, store, namespace=underlying_embedder.model
)

In [4]:
%%time
embeddings = embedder.embed_documents(["hello", "goodbye"])

CPU times: user 405 ms, sys: 32.9 ms, total: 438 ms
Wall time: 715 ms


The second time we try to embed the embedding time is only 2 ms because the embeddings are looked up in the cache.

In [5]:
%%time
embeddings_from_cache = embedder.embed_documents(["hello", "goodbye"])

CPU times: user 1.55 ms, sys: 436 µs, total: 1.99 ms
Wall time: 1.99 ms


In [6]:
embeddings == embeddings_from_cache

True

## File system

This section covers how to use a file system store

In [7]:
fs = LocalFileStore("./cache/")

In [8]:
embedder2 = CacheBackedEmbedder.from_bytes_store(
    underlying_embedder, fs, namespace=underlying_embedder.model
)

In [9]:
%%time
embeddings = embedder2.embed_documents(["hello", "goodbye"])

CPU times: user 10.5 ms, sys: 988 µs, total: 11.5 ms
Wall time: 220 ms


In [10]:
%%time
embeddings = embedder2.embed_documents(["hello", "goodbye"])

CPU times: user 3.49 ms, sys: 0 ns, total: 3.49 ms
Wall time: 3.03 ms


Here are the embeddings that have been persisted to the directory `./cache`. 

Notice that the embedder takes a namespace parameter.

In [11]:
list(fs.yield_keys())

['text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5',
 'text-embedding-ada-0026ba52e44-59c9-5cc9-a084-284061b13c80']

## Using with a vectorstore

Let's see this cache in action with the FAISS vectorstore.

In [12]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [13]:
fs = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbedder.from_bytes_store(
    OpenAIEmbeddings(), fs, namespace=underlying_embedder.model
)

The cache is empty prior to embedding

In [14]:
list(fs.yield_keys())

['text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5',
 'text-embedding-ada-0026ba52e44-59c9-5cc9-a084-284061b13c80']

Load the document, split it into chunks, embed each chunk and load it into the vector store.

In [16]:
raw_documents = TextLoader("../state_of_the_union.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

create the vectorstore

In [17]:
%%time
db = FAISS.from_documents(documents, cached_embedder)

CPU times: user 124 ms, sys: 22.6 ms, total: 146 ms
Wall time: 832 ms


If we try to create the vectostore again, it'll be much faster since it does not need to re-compute any embeddings.

In [18]:
%%time
db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 32.9 ms, sys: 286 µs, total: 33.2 ms
Wall time: 32.5 ms


And here are some of the embeddings that got created:

In [19]:
list(fs.yield_keys())[:5]

['text-embedding-ada-002614d7cf6-46f1-52fa-9d3a-740c39e7a20e',
 'text-embedding-ada-0020fc1ede2-407a-5e14-8f8f-5642214263f5',
 'text-embedding-ada-002e885db5b-c0bd-5fbc-88b1-4d1da6020aa5',
 'text-embedding-ada-002e4ad20ef-dfaa-5916-9459-f90c6d8e8159',
 'text-embedding-ada-002a5ef11e4-0474-5725-8d80-81c91943b37f']